# Lecture4: Confidence Intervals/ Statistical Inference

- Normal Distributions and theotretical populations
- Samling
- Bootstrapping
- Confindence intervals

In [1]:
# Import Dataset

import pandas as pd
url = ("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv")
penguins_raw = pd.read_csv(url) #avatar.isnull().sum() #avatar[avatar.isnull().sum(axis=1)>0]

print(penguins_raw.shape)
penguins_raw.head()

(344, 7)


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


In [2]:
# Drop the nan values and those which we can not work with
penguins_noNaN = penguins_raw.drop(['species', 'island', 'bill_length_mm', 'bill_depth_mm', 'sex'], axis=1).dropna()

penguins_noNaN['flipper_length_mm'] = penguins_noNaN['flipper_length_mm'].astype(int)
penguins_noNaN['body_mass_g'] = penguins_noNaN['body_mass_g'].astype(int)
print(penguins_noNaN.shape)
penguins_noNaN.dtypes

(342, 2)


flipper_length_mm    int64
body_mass_g          int64
dtype: object

## Theoretical Populations

In [3]:
import plotly.express as px
from scipy import stats
import numpy as np


# how to draw a theoretical populations

mu, sigma = 1, 0.33


my_theoretical_population = stats.norm(loc=mu, scale=sigma)

support = np.linspace(-1, 3, 100)

df = pd.DataFrame({'x': support, 'density': my_theoretical_population.pdf(support)})

px.line(df, x='x', y='density')

### Now lets sample from it and draw this population again

In [4]:
# Example of sampling
my_theoretical_population.rvs(size=30)

array([1.22513442, 0.99294215, 0.82316552, 1.03762876, 0.76933388,
       1.37333679, 0.80751878, 0.80743418, 0.86819938, 1.35128829,
       0.76702856, 0.92785315, 0.82105642, 1.30912345, 0.85888246,
       1.05508211, 0.47780746, 0.62743138, 1.26642112, 0.77621473,
       0.88795531, 0.18507377, 0.9838883 , 1.00706959, 0.76654078,
       1.28621999, 1.54604939, 0.88500132, 1.16243879, 0.6142086 ])

### Now we want to do this sampling but in repeat.

In [5]:
# Sampling from the theoretical populations

# how many times we want to do this
reps = 1000

# what is our sample size
sample_size = 333

# make an array to save the values
my_theoretical_population_sample_means = np.zeros(reps)

# make a random seed
np.random.seed(130)

# loop through in that many times
for i in range(reps):

    # take a sample from the population
    my_theoretical_sample = my_theoretical_population.rvs(size=sample_size)

    # calculate the mean and add it to the array
    my_theoretical_population_sample_means[i] = my_theoretical_sample.mean()

# make a dataframe of the bootstrapped samples
df = pd.DataFrame({'sample_averages': my_theoretical_population_sample_means})

# graph
px.histogram(df, x='sample_averages')

Lets make a null-hypothesis

$H_0:$ The true mean and the mean of the theoretical population are the same

$H_1:$ They should not be the same

In [15]:
# if we graph them on each other we get

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Histogram(x=my_theoretical_population_sample_means, histnorm='probability density', nbinsx=20))
fig.add_vline(x= my_theoretical_population.mean())

fig.show()

We can not reject the null hypothesis as the distribution is symetric around 1

## Now lets apply this to the data

#### The peace of code you just wrote is called bootstrapping.

Bootstrapping calculates a statistic for us and to make sure we will have this for the true population we generate random samples and then we calculate this statistic. If we do this enough times, then we will get closer to the true population.

In [7]:
# Bootstrapped Sampling from the sample

# how many times we want to do this
reps = 1000

# what is our sample size
sample_size = 342

# make an array to save the values
my_bootstrapped_population_sample_means = np.zeros(reps)

# make a random seed
np.random.seed(130)

# loop through in that many times
for i in range(reps):

    # take a sample from the population
    my_bootstrapped_sample = np.random.choice(penguins_noNaN.body_mass_g, size=sample_size, replace=True)

    # calculate the mean and add it to the array
    my_bootstrapped_population_sample_means[i] = my_bootstrapped_sample.mean()

In [8]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=my_bootstrapped_population_sample_means, histnorm='probability density', nbinsx=20))
fig.show()

Now lets make a null hypothesis:

$H_0:$ The true mean is 4000 # Chosen randomly

$H_1:$ The true mean is not 4000


Now Since Bootstrapping works 95 percent fo the time, to make sure that we deal with his error, we gonna do a 95 percent confidence interval where we will select teh middle 95 percent fo the bootstrapped data

In [9]:
# 95 precent confidence interval
data_quantile = np.quantile(my_bootstrapped_population_sample_means, [0.025,0.975 ]) # to get the mass
data_quantile

array([4117.23135965, 4284.27997076])

In [17]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=my_bootstrapped_population_sample_means, histnorm='probability density', nbinsx=20))
fig.add_vline(x=data_quantile[0], line_dash='dash', line_color='white')
fig.add_vline(x=data_quantile[1], line_dash='dash', line_color='white')
fig.add_vline(x=4000)
fig.show()

Therefore we can obviously reject the null hypothesis since it is not in the confidence internval

### Optional practice: now do this with the `flipper_length_mm` column

In [11]:
# Bootstrapped Sampling from the sample

# how many times we want to do this
reps = 1000

# what is our sample size
sample_size = 342

# make an array to save the values
my_bootstrapped_population_sample_means_2 = np.zeros(reps)

# make a random seed
np.random.seed(130)

# loop through in that many times
for i in range(reps):

    # take a sample from the population
    my_bootstrapped_sample = np.random.choice(penguins_noNaN.flipper_length_mm, size=sample_size, replace=True)

    # calculate the mean and add it to the array
    my_bootstrapped_population_sample_means_2[i] = my_bootstrapped_sample.mean()# This is so we get a ratio in the same dimentions

In [12]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=my_bootstrapped_population_sample_means_2, histnorm='probability density', nbinsx=20))
fig.show()